### Imports and installs

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json

from datasets import Dataset
from sklearn.metrics import confusion_matrix
import time

### Uniform dataset creation

In [ ]:
f_control = open('control_with_contribs.json')
control_users = json.load(f_control).keys()

f_sock = open('socks_with_contribs.json')
sock_users = json.load(f_sock).keys()

In [ ]:
def describe_sets(y_train, y_test, set_nr):
    print("Describing dataset: " + str(set_nr))
    train_socks = 0
    train_control = 0
    for y in y_train:
        if y == 'sock':
            train_socks += 1
        else:
            train_control += 1
            
    print("Socks in training set: " + str(train_socks))
    print("Control in training set: " + str(train_control))
    
    
    test_socks = 0
    test_control = 0
    for y in y_test:
        if y == 'sock':
            test_socks += 1
        else:
            test_control += 1
            
    print("Socks in testing set: " + str(test_socks))
    print("Control in testing set: " + str(test_control))
    print("------------------------------------------")

In [ ]:
X = list(sock_users) + list(control_users)
if not len(X) == len(sock_users) + len(control_users):
    print("ERROR")
y = ['sock'] * len(sock_users) + ['control'] * len(control_users)
if not len(y) == len(sock_users) + len(control_users):
    print('ERROR')

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.3, random_state=0)
describe_sets(y_train1, y_test1, 1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.3, random_state=11)
describe_sets(y_train2, y_test2, 2)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.3, random_state=33)
describe_sets(y_train3, y_test3, 3)

Describing dataset: 1
Socks in training set: 1756
Control in training set: 1720
Socks in testing set: 727
Control in testing set: 763
------------------------------------------
Describing dataset: 2
Socks in training set: 1721
Control in training set: 1755
Socks in testing set: 762
Control in testing set: 728
------------------------------------------
Describing dataset: 3
Socks in training set: 1736
Control in training set: 1740
Socks in testing set: 747
Control in testing set: 743
------------------------------------------


In [ ]:
f_control = open('control_with_contribs_MODIFIED_FORMAT.json')
control_data = pd.read_json(f_control)

f_sock = open('socks_with_contribs_MODIFIED_FORMAT.json')
sock_data = pd.read_json(f_sock)

In [ ]:
all_data = control_data.append(sock_data)

<ipython-input-9-77f91764d601>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = control_data.append(sock_data)


In [ ]:
def get_alligned_train_and_test_data(all_data, X_users, y_users):
  source = all_data[all_data["author"].isin(X_users)]
  data_df = source.loc[source["comments"] != ""]
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))
  data_df = data_df[['comments', "author"]].drop_duplicates()

  assert len(X_users) == len(y_users)

  user_to_label = {}
  for i in range(0, len(X_users)):
    user_to_label[X_users[i]] = y_users[i]

  users_order = data_df["author"]
  print(type(users_order))

  labels = []

  for user in users_order:
    label = 1 if user_to_label[user] == "sock" else 0
    labels.append(label)

  data_df["label"] = labels
  
  data_df = data_df.drop(columns=["author"])

  return data_df
  

In [ ]:
train_example = get_alligned_train_and_test_data(all_data, X_train1, y_train1)

<ipython-input-9-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


In [ ]:
train_example

,comments,label
2,"== ""Bionicle in Japan"" listed at [[Wikipedia:R...",0
3,==Early years== I have removed the quotes from...,0
4,I've been a student at Kenji Tomiki's dojo in ...,0
5,{{Old prod full | nom = 115.69.17.225 | ...,0
6,== Needs Editing == I agree some bands in this...,0
...,...,...
40075,:::cobra your patience and pedagogy are admira...,1
40077,{{Talk header}} {{blpo}} {{Film|class=Start|In...,1
40078,== Contested deletion == This page is not unam...,1
40079,== Kashmir == During the Kashmir section (or r...,1


In [ ]:
sum(train_example["label"])

1612

### Load dataset

In [ ]:
# aggregated_sock_data = sock_data.loc[sock_data["comments"] != ""]
# aggregated_sock_data["comments"] = aggregated_sock_data.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))
# aggregated_sock_data = aggregated_sock_data[['comments', "author"]].drop_duplicates()
# aggregated_sock_data["label"] = [1] * len(aggregated_sock_data)

# aggregated_control_data = control_data.loc[control_data["comments"] != ""]
# aggregated_control_data["comments"] = aggregated_control_data.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))
# aggregated_control_data = aggregated_control_data[['comments', "author"]].drop_duplicates()
# aggregated_control_data["label"] = [0] * len(aggregated_control_data)

In [ ]:
# all_data_df = aggregated_control_data.append(aggregated_sock_data)

# all_data_df

In [ ]:
# dataset = Dataset.from_pandas(all_data_df, split = ['train', 'validation', 'test'])

In [ ]:
# X1, X_test = train_test_split(all_data_df, test_size = 0.2, random_state = 10)
# X_train, X_validation = train_test_split(X1, test_size = 0.3, random_state = 10)

In [ ]:
# train_data = Dataset.from_pandas(X_train)
# validation_data = Dataset.from_pandas(X_validation)
# test_data = Dataset.from_pandas(X_test)

### Functions for all transformers

In [ ]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text["comments"], padding = True, truncation=True)

In [ ]:
# define accuracy metrics
def compute_metrics(pred):
    labelsai = pred.label_ids
    print(len(labelsai))
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labelsai, preds, average='binary')
    acc = accuracy_score(labelsai, preds)
    tn, fp, fn, tp = confusion_matrix(labelsai, preds, labels = [0, 1]).ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (tn + fp)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'tpr(recall)':tpr,
        'fpr':fpr
    }

In [ ]:
all_results = {"Roberta" : {"TPR" : [], "FPR" : [], "Precision": [], "F-measure" : [], "Train_time" : [], "Test_time" : []},
                     "Distilbert" : {"TPR" : [], "FPR" : [], "Precision": [], "F-measure" : [], "Train_time" : [], "Test_time" : []},
                     "Bert" : {"TPR" : [], "FPR" : [], "Precision": [], "F-measure" : [], "Train_time" : [], "Test_time" : []},
                     "XLNet" : {"TPR" : [], "FPR" : [], "Precision": [], "F-measure" : [], "Train_time" : [], "Test_time" : []}}
               
def fill_in_results(model, results_dict, test_time, train_time):
    tpr = results_dict["eval_tpr(recall)"]
    fpr = results_dict["eval_fpr"]
    precision = results_dict["eval_precision"]
    f_score = results_dict["eval_f1"]
    all_results[model]["TPR"].append(tpr)
    all_results[model]["FPR"].append(fpr)
    all_results[model]["Precision"].append(precision)
    all_results[model]["F-measure"].append(f_score)
    all_results[model]["Train_time"].append(train_time)
    all_results[model]["Test_time"].append(test_time)
    

### Model - RoBERTa

In [ ]:
for data in [(X_train1, X_test1, y_train1, y_test1), (X_train2, X_test2, y_train2, y_test2), (X_train3, X_test3, y_train3, y_test3)]:
    X_train_users = data[0]
    X_test_users = data[1]
    y_train_users = data[2] 
    y_test_users = data[3]
    X_both = get_alligned_train_and_test_data(all_data, X_train_users, y_train_users)
    X_train, X_validation = train_test_split(X_both, test_size = 0.3, random_state = 10)
    X_test = get_alligned_train_and_test_data(all_data, X_test_users, y_test_users)
    train_data = Dataset.from_pandas(X_train)
    validation_data = Dataset.from_pandas(X_validation)
    test_data = Dataset.from_pandas(X_test)

    # load model and tokenizer and define length of the text sequence
    model = RobertaForSequenceClassification.from_pretrained('roberta-base')
    tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

    train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
    validation_data = validation_data.map(tokenization, batched = True, batch_size = len(validation_data))
    test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

    train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    validation_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    # define the training arguments
    training_args = TrainingArguments(
        output_dir = '/results',
        num_train_epochs=3,
        per_device_train_batch_size = 4,   
        per_device_eval_batch_size= 8,
        warmup_steps=500,                
        weight_decay=0.01,               
        logging_dir='./logs',            
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=validation_data
    )

    # train the model
    train_start = time.time()
    trainer.train()
    train_total = time.time() - train_start

    test_start = time.time()
    results = trainer.evaluate(test_data)
    test_total = time.time() - test_start

    fill_in_results("Roberta", results, test_total, train_total)



<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

...

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Map:   0%|          | 0/1372 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.701100
20,0.711200
30,0.713300
40,0.709100
50,0.675100
60,0.688400
70,0.720600
80,0.685100
90,0.698200
100,0.683800


1372


### Model - DistilBERT

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bia

In [ ]:
for data in [(X_train1, X_test1, y_train1, y_test1), (X_train2, X_test2, y_train2, y_test2), (X_train3, X_test3, y_train3, y_test3)]:
    X_train_users = data[0]
    X_test_users = data[1]
    y_train_users = data[2] 
    y_test_users = data[3]
    X_both = get_alligned_train_and_test_data(all_data, X_train_users, y_train_users)
    X_train, X_validation = train_test_split(X_both, test_size = 0.3, random_state = 10)
    X_test = get_alligned_train_and_test_data(all_data, X_test_users, y_test_users)
    train_data = Dataset.from_pandas(X_train)
    validation_data = Dataset.from_pandas(X_validation)
    test_data = Dataset.from_pandas(X_test)

    # load model and tokenizer and define length of the text sequence
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased")
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

    train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
    validation_data = validation_data.map(tokenization, batched = True, batch_size = len(validation_data))
    test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

    train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    validation_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    # define the training arguments
    training_args = TrainingArguments(
        output_dir = '/results',
        num_train_epochs=3,
        per_device_train_batch_size = 4,   
        per_device_eval_batch_size= 8,
        warmup_steps=500,                
        weight_decay=0.01,               
        logging_dir='./logs',            
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=validation_data
    )

    # train the model
    train_start = time.time()
    trainer.train()
    train_total = time.time() - train_start

    test_start = time.time()
    results = trainer.evaluate(test_data)
    test_total = time.time() - test_start

    fill_in_results("Distilbert", results, test_total, train_total)



<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bia

...

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Map:   0%|          | 0/1372 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.691200
20,0.710900
30,0.694000
40,0.708300
50,0.703100
60,0.703400
70,0.701100
80,0.688300
90,0.700800
100,0.697200


1372


### Model - BERT

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

In [ ]:
for data in [(X_train1, X_test1, y_train1, y_test1), (X_train2, X_test2, y_train2, y_test2), (X_train3, X_test3, y_train3, y_test3)]:
    X_train_users = data[0]
    X_test_users = data[1]
    y_train_users = data[2] 
    y_test_users = data[3]
    X_both = get_alligned_train_and_test_data(all_data, X_train_users, y_train_users)
    X_train, X_validation = train_test_split(X_both, test_size = 0.3, random_state = 10)
    X_test = get_alligned_train_and_test_data(all_data, X_test_users, y_test_users)
    train_data = Dataset.from_pandas(X_train)
    validation_data = Dataset.from_pandas(X_validation)
    test_data = Dataset.from_pandas(X_test)

    # load model and tokenizer and define length of the text sequence
    model = BertForSequenceClassification.from_pretrained('bert-base-cased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

    train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
    validation_data = validation_data.map(tokenization, batched = True, batch_size = len(validation_data))
    test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

    train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    validation_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    # define the training arguments
    training_args = TrainingArguments(
        output_dir = '/results',
        num_train_epochs=3,
        per_device_train_batch_size = 4,   
        per_device_eval_batch_size= 8,
        warmup_steps=500,                
        weight_decay=0.01,               
        logging_dir='./logs',            
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=validation_data
    )

    # train the model
    train_start = time.time()
    trainer.train()
    train_total = time.time() - train_start

    test_start = time.time()
    results = trainer.evaluate(test_data)
    test_total = time.time() - test_start

    fill_in_results("Bert", results, test_total, train_total)



<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


<ipython-input-26-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

...

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Map:   0%|          | 0/1372 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.735800
20,0.716100
30,0.668100
40,0.701100
50,0.677700
60,0.657600
70,0.664000
80,0.700400
90,0.719300
100,0.684600


1372


### Model - XLNet

In [ ]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer
from transformers import XLNetTokenizer, XLNetModel

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
for data in [(X_train1, X_test1, y_train1, y_test1), (X_train2, X_test2, y_train2, y_test2), (X_train3, X_test3, y_train3, y_test3)]:
    X_train_users = data[0]
    X_test_users = data[1]
    y_train_users = data[2] 
    y_test_users = data[3]
    X_both = get_alligned_train_and_test_data(all_data, X_train_users, y_train_users)
    X_train, X_validation = train_test_split(X_both, test_size = 0.3, random_state = 10)
    X_test = get_alligned_train_and_test_data(all_data, X_test_users, y_test_users)
    train_data = Dataset.from_pandas(X_train)
    validation_data = Dataset.from_pandas(X_validation)
    test_data = Dataset.from_pandas(X_test)

    # load model and tokenizer and define length of the text sequence
    model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
    tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', model_max_length=512)

    train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
    validation_data = validation_data.map(tokenization, batched = True, batch_size = len(validation_data))
    test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

    train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    validation_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    # define the training arguments
    training_args = TrainingArguments(
        output_dir = '/results',
        num_train_epochs=3,
        per_device_train_batch_size = 4,   
        per_device_eval_batch_size= 8,
        warmup_steps=500,                
        weight_decay=0.01,               
        logging_dir='./logs',            
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=validation_data
    )

    # train the model
    train_start = time.time()
    trainer.train()
    train_total = time.time() - train_start

    test_start = time.time()
    results = trainer.evaluate(test_data)
    test_total = time.time() - test_start

    fill_in_results("XLNet", results, test_total, train_total)



<ipython-input-10-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


<ipython-input-10-28897190389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["comments"] = data_df.groupby(['author'])['comments'].transform(lambda x: ' '.join(x))


<class 'pandas.core.series.Series'>


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Map:   0%|          | 0/1372 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.717000
20,0.694200
30,0.692700
40,0.721300
50,0.647900
60,0.626600
70,0.662500
80,0.641300
90,0.742000
100,0.755300


1372


### Results

In [ ]:
# all_results

In [ ]:
for model, results in all_results.items():
    print("--------------------------")
    print(str(model) + " RESULTS: ")
    iteration_dict = results.copy()
    for metric, res in iteration_dict.items():  
        print(metric)
        average = sum(res)/len(res)
        name = metric + " AVG"
        all_results[model][name] = average
        print("M = " + str(average))
        std = np.std(res)
        name = metric + " STD"
        all_results[model][name] = std
        print("STD = " + str(std))
          

In [ ]:
# save results to excel:
all_results_df = pd.DataFrame(data=all_results)
all_results_df = all_results_df.T
all_results_df.to_excel('transformer_results.xlsx')
all_results_df

,TPR,FPR,Precision,F-measure,Train_time,Test_time,TPR AVG,TPR STD,FPR AVG,FPR STD,Precision AVG,Precision STD,F-measure AVG,F-measure STD,Train_time AVG,Train_time STD,Test_time AVG,Test_time STD
Roberta,"[0.8582202111613876, 0.8347701149425287, 0.837...","[0.10235131396957123, 0.08454810495626822, 0.1...","[0.8849144634525661, 0.9092331768388107, 0.884...","[0.8713629402756509, 0.8704119850187266, 0.860...","[737.4092082977295, 738.357971906662, 740.0493...","[42.47421145439148, 42.456140756607056, 41.853...",0.843503,0.010467,0.097791,0.009515,0.892901,0.01155,0.86739,0.004962,738.605498,1.091938,42.261345,0.288356
Distilbert,"[0.755656108597285, 0.7514367816091954, 0.7902...","[0.2835408022130014, 0.2696793002915452, 0.282...","[0.7096317280453258, 0.7387005649717514, 0.731...","[0.7319211102994887, 0.7450142450142448, 0.759...","[374.4829988479614, 369.8833591938019, 371.338...","[22.84773898124695, 22.778496980667114, 22.561...",0.765781,0.017388,0.278411,0.006206,0.726735,0.012411,0.745626,0.011448,371.901568,1.919564,22.729273,0.121897
Bert,"[0.7496229260935143, 0.7442528735632183, 0.737...","[0.2600276625172891, 0.3032069970845481, 0.258...","[0.7255474452554744, 0.7134986225895317, 0.734...","[0.7373887240356084, 0.7285513361462729, 0.735...","[724.5583093166351, 726.308119058609, 728.7823...","[45.700156688690186, 45.46646285057068, 44.961...",0.74365,0.00514,0.274076,0.020603,0.72465,0.008762,0.733976,0.003878,726.549605,1.732906,45.375984,0.308334
XLNet,"[0.7481146304675717, 0.17672413793103448, 0.97...","[0.35269709543568467, 0.0728862973760933, 0.72...","[0.6604527296937417, 0.7109826589595376, 0.565...","[0.7015558698727016, 0.28308400460299193, 0.71...","[1320.1769676208496, 1319.7746748924255, 1324....","[140.02017378807068, 139.1446177959442, 138.46...",0.632258,0.334821,0.384547,0.268419,0.645576,0.060401,0.56649,0.200471,1321.469092,2.11818,139.208414,0.638351
